### Convert Aggregate Data (current version) to Time-Series for more Granularity

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [21]:
data = pd.read_csv('phase2data_edited.csv', index_col=False)
data

,Unnamed: 0,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,W,L,%,P_W,P_L,P_%,URL
0,0,174258.0,"Brown, Gary",11.0,6.0,7/1/1969 0:00:00,8.0,3.0,RB,0.0,Off,NaN,83.0,93.0,0.472,2.0,3.0,0.400,https://pro-football-history.com/coach/861/gar...
1,1,660434.0,"Levine, Tony",2.0,18.0,10/28/1972 0:00:00,0.0,3.0,OC,0.0,ST,NaN,15.0,17.0,0.469,0.0,0.0,0.000,https://pro-football-history.com/coach/1177/to...
2,2,580829.0,"Scherer, Rip",9.0,30.0,8/3/1952 0:00:00,0.0,3.0,HC,0.0,Off,NaN,58.0,86.0,0.403,1.0,1.0,0.500,https://pro-football-history.com/coach/1196/ri...
3,3,154631.0,"Drevno, Tim",3.0,29.0,3/20/1969 0:00:00,0.0,3.0,QC,0.0,Off,NaN,36.0,11.0,0.760,5.0,3.0,0.625,https://pro-football-history.com/coach/1315/ti...
4,4,926678.0,"Senger, Connor",2.0,5.0,NaN,0.0,3.0,OC,0.0,Off,NaN,8.0,26.0,0.235,0.0,0.0,0.000,https://pro-football-history.com/coach/13872/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2273,499900.0,"Skipper, Kelly",17.0,18.0,7/25/1967 0:00:00,0.0,3.0,OC,0.0,Off,NaN,129.0,161.0,0.445,5.0,6.0,0.455,https://pro-football-history.com/coach/2259/ke...
2274,2274,470114.0,"Rodgers, Kacy",21.0,9.0,6/24/1969 0:00:00,0.0,2.0,AH,4.0,Def,NaN,173.0,166.0,0.510,6.0,7.0,0.461,https://pro-football-history.com/coach/841/kac...
2275,2275,881149.0,"Lester, Tim",1.0,22.0,2/8/1977 0:00:00,0.0,3.0,HC,0.0,Off,NaN,9.0,8.0,0.529,1.0,1.0,0.500,https://pro-football-history.com/coach/14053/t...
2276,2276,135716.0,"Faulkner, Eddie",5.0,17.0,10/20/1977 0:00:00,1.0,3.0,OC,0.0,NaN,NaN,48.0,34.0,0.584,0.0,3.0,0.000,https://pro-football-history.com/coach/12683/e...


In [22]:
data.values.tolist()[0]

[0,
 174258.0,
 'Brown, Gary',
 11.0,
 6.0,
 '7/1/1969 0:00:00',
 8.0,
 3.0,
 'RB',
 0.0,
 'Off',
 nan,
 83.0,
 93.0,
 0.472,
 2.0,
 3.0,
 0.4,
 'https://pro-football-history.com/coach/861/gary-brown-bio']

In [23]:
list_data = [row[1:] for row in data.values.tolist()]
list_data[0]

[174258.0,
 'Brown, Gary',
 11.0,
 6.0,
 '7/1/1969 0:00:00',
 8.0,
 3.0,
 'RB',
 0.0,
 'Off',
 nan,
 83.0,
 93.0,
 0.472,
 2.0,
 3.0,
 0.4,
 'https://pro-football-history.com/coach/861/gary-brown-bio']

In [10]:
# get data from url
def get_data(url):
    soup = BeautifulSoup(urlopen(url), features='lxml')
    rows = soup.findAll('tr')[1:]
    rows_data = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    return rows_data

In [25]:
time_series_data = []

In [37]:
for i in range(len(list(data.index))):
    url = data.iloc[i]['URL']
    print(url)
    if url == None:
        continue
    coach_data = get_data(url)[2:-1]
    
    coach_data.reverse()
    coach_data = [[element.strip() for element in coach] for coach in coach_data]
    coach_data = [coach[0].split('  ')+coach[1:] for coach in coach_data]
    coach_data = [coach for coach in coach_data if len(coach) == 5]
    coach_data = [coach[:3] + coach[3].split('-') + coach[4].split('-') for coach in coach_data]

    coach_bio = list_data[i][:11]+[list_data[i][-1]]
    coach_cume = []
    for row in coach_data:
        coach_cume.append(coach_bio+row)
    
    time_series_data += coach_cume

https://pro-football-history.com/coach/861/gary-brown-bio
https://pro-football-history.com/coach/1177/tony-levine-bio
https://pro-football-history.com/coach/1196/rip-scherer-bio
https://pro-football-history.com/coach/1315/tim-drevno-bio


HTTPError: HTTP Error 503: Service Temporarily Unavailable

In [27]:
for row in time_series_data:
    print(row)

[174258.0, 'Brown, Gary', 11.0, 6.0, '7/1/1969 0:00:00', 8.0, 3.0, 'RB', 0.0, 'Off', nan, 'https://pro-football-history.com/coach/861/gary-brown-bio', '2009', 'Cleveland Browns', 'Running Backs Coach', '5', '11', '0', '0', '0']
[174258.0, 'Brown, Gary', 11.0, 6.0, '7/1/1969 0:00:00', 8.0, 3.0, 'RB', 0.0, 'Off', nan, 'https://pro-football-history.com/coach/861/gary-brown-bio', '2010', 'Cleveland Browns', 'Running Backs Coach', '5', '11', '0', '0', '0']
[174258.0, 'Brown, Gary', 11.0, 6.0, '7/1/1969 0:00:00', 8.0, 3.0, 'RB', 0.0, 'Off', nan, 'https://pro-football-history.com/coach/861/gary-brown-bio', '2011', 'Cleveland Browns', 'Running Backs Coach', '4', '12', '0', '0', '0']
[174258.0, 'Brown, Gary', 11.0, 6.0, '7/1/1969 0:00:00', 8.0, 3.0, 'RB', 0.0, 'Off', nan, 'https://pro-football-history.com/coach/861/gary-brown-bio', '2012', 'Cleveland Browns', 'Running Backs Coach', '5', '11', '0', '0', '0']
[174258.0, 'Brown, Gary', 11.0, 6.0, '7/1/1969 0:00:00', 8.0, 3.0, 'RB', 0.0, 'Off', nan

In [29]:
list(data.columns)[1:]

['CoachID',
 'Name',
 'NFL Coach Experience (thru 2023)',
 'College/Other Exp (thru 2023)',
 'Date of Birth',
 'NFL Playing Exp',
 'Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)',
 'Highest Pos College / other Pro',
 '# seasons as NFL Coordinator',
 'Off/Def side of ball',
 '1st Season as NFL HC',
 'W',
 'L',
 '%',
 'P_W',
 'P_L',
 'P_%',
 'URL']

In [31]:
time_series_df = pd.DataFrame(time_series_data, columns=list(data.columns)[1:-7]+['URL']+['Season','Team','Pos','W','L','T','P_W','P_L'])

In [32]:
time_series_df

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,URL,Season,Team,Pos,W,L,T,P_W,P_L
0,174258.0,"Brown, Gary",11.0,6.0,7/1/1969 0:00:00,8.0,3.0,RB,0.0,Off,NaN,https://pro-football-history.com/coach/861/gar...,2009,Cleveland Browns,Running Backs Coach,5,11,0,0,0
1,174258.0,"Brown, Gary",11.0,6.0,7/1/1969 0:00:00,8.0,3.0,RB,0.0,Off,NaN,https://pro-football-history.com/coach/861/gar...,2010,Cleveland Browns,Running Backs Coach,5,11,0,0,0
2,174258.0,"Brown, Gary",11.0,6.0,7/1/1969 0:00:00,8.0,3.0,RB,0.0,Off,NaN,https://pro-football-history.com/coach/861/gar...,2011,Cleveland Browns,Running Backs Coach,4,12,0,0,0
3,174258.0,"Brown, Gary",11.0,6.0,7/1/1969 0:00:00,8.0,3.0,RB,0.0,Off,NaN,https://pro-football-history.com/coach/861/gar...,2012,Cleveland Browns,Running Backs Coach,5,11,0,0,0
4,174258.0,"Brown, Gary",11.0,6.0,7/1/1969 0:00:00,8.0,3.0,RB,0.0,Off,NaN,https://pro-football-history.com/coach/861/gar...,2013,Dallas Cowboys,Running Backs Coach,8,8,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,323691.0,"Billick, Brian",16.0,12.0,2/28/1954 0:00:00,1.0,1.0,OC,5.0,Off,1999.0,https://pro-football-history.com/coach/32/bria...,2003,Baltimore Ravens,Head Coach,10,6,0,0,1
81,323691.0,"Billick, Brian",16.0,12.0,2/28/1954 0:00:00,1.0,1.0,OC,5.0,Off,1999.0,https://pro-football-history.com/coach/32/bria...,2004,Baltimore Ravens,Head Coach,9,7,0,0,0
82,323691.0,"Billick, Brian",16.0,12.0,2/28/1954 0:00:00,1.0,1.0,OC,5.0,Off,1999.0,https://pro-football-history.com/coach/32/bria...,2005,Baltimore Ravens,Head Coach,6,10,0,0,0
83,323691.0,"Billick, Brian",16.0,12.0,2/28/1954 0:00:00,1.0,1.0,OC,5.0,Off,1999.0,https://pro-football-history.com/coach/32/bria...,2006,Baltimore Ravens,Head Coach,13,3,0,0,1
